In [1]:
import cv2
import numpy as np
import time
import sys
import os

def calculate_iou(box1, box2):
    # Извлекаем координаты прямоугольников
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Вычисляем координаты углов
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)

    # Вычисляем площадь пересечения
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

    # Вычисляем площади прямоугольников
    area1 = w1 * h1
    area2 = w2 * h2

    # Вычисляем коэффициент пересечения ограничивающих прямоугольников (IoU)
    iou = intersection_area / float(area1 + area2 - intersection_area)

    return iou


CONFIDENCE = 0.7
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

# конфигурация нейронной сети
config_path = "cfg/yolov3.cfg"
# файл весов сети YOLO
weights_path = "weights/yolov3.weights"
# weights_path = "weights/yolov3-tiny.weights"
# загрузка всех меток классов (объектов)
labels = open("data/coco.names").read().strip().split("\n")
# генерируем цвета для каждого объекта и последующего построения
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
path_name = "pic1.png"
image = cv2.imread(path_name)
file_name = os.path.basename(path_name)
filename, ext = file_name.split(".")
h, w = image.shape[:2]
# создать 4D blob
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True,crop=False)
print("image.shape:", image.shape)
print("blob.shape:", blob.shape)
net.setInput(blob)
# получаем имена всех слоев
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

# прямая связь (вывод) и получение выхода сети
# измерение времени для обработки в секундах
start = time.perf_counter()
layer_outputs = net.forward(ln)
time_took = time.perf_counter() - start
print(f"Потребовалось: {time_took:.2f}s")
font_scale = 1
thickness = 2
boxes, confidences, class_ids = [], [], []
# перебираем каждый из выходов слоя
for output in layer_outputs:
    # перебираем каждое обнаружение объекта
    for detection in output:
        # извлекаем идентификатор класса (метку) и достоверность (как вероятность)
        # обнаружение текущего объекта
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        #пропуск слабых прогнозов
        if confidence > CONFIDENCE:
            # масштабируем координаты ограничивающего прямоугольника относительно размера изображения, учитывая, что YOLO возвращает центр поля
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            # используем центры (x, y) для получения левого верхнего угла
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

cars = []
bicycles = []

# Перебираем обнаруженные объекты и добавляем их в соответствующие списки
for i in range(len(boxes)):
    class_id = class_ids[i]
    label = labels[class_id]
    confidence = confidences[i]
    if label == 'car':
        # Проверяем, не пересекается ли текущий объект с уже добавленными машинами
        is_intersecting = False
        for existing_car in cars:
            if calculate_iou(boxes[i], existing_car[0]) > 0.50:
                is_intersecting = True
                break
        if not is_intersecting:
            cars.append((boxes[i], confidence))
    elif label == 'bicycle':
        # Проверяем, не пересекается ли текущий объект с уже добавленными велосипедами
        is_intersecting = False
        for existing_bicycle in bicycles:
            if calculate_iou(boxes[i], existing_bicycle[0]) > 0.50:
                is_intersecting = True
                break
        if not is_intersecting:
            bicycles.append((boxes[i], confidence))

# выполнить не максимальное подавление с учетом оценок, определенных ранее
idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)


# Сортируем списки объектов по расстоянию от камеры
cars.sort(key=lambda x: x[0][2] * x[0][3], reverse=True)  # По площади ограничивающего прямоугольника
bicycles.sort(key=lambda x: x[0][2] * x[0][3], reverse=True)  # По площади ограничивающего прямоугольника

# убедитесь, что обнаружен хотя бы один объект
if len(idxs) > 0:
    # инициализируем счетчик объектов
    i = 0
    
    # перебираем сохраняемые индексы
    # Выводим на изображение отсортированные объекты
    for (box, confidence) in cars:
        i += 1  # увеличиваем счетчик объектов
        x, y, w, h = box
        color = [int(c) for c in colors[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
        text = f"car {i}: {confidence:.2f}"  # добавляем порядковый номер объекта
        cv2.putText(image, text, (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=font_scale, color=(0, 0, 255), thickness=thickness)
    i=0
    for (box, confidence) in bicycles:
        i += 1  # увеличиваем счетчик объектов
        x, y, w, h = box
        color = [int(c) for c in colors[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
        text = f"bicycle {i}: {confidence:.2f}"  # добавляем порядковый номер объекта
        cv2.putText(image, text, (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=font_scale, color=(0, 0, 255), thickness=thickness)
                    
# сохраняем изображение с отрисованными объектами
cv2.imwrite(filename + "_yolo3_sorted." + ext, image)




image.shape: (1800, 2400, 3)
blob.shape: (1, 3, 416, 416)
Потребовалось: 1.75s


True